In this notebook we will experiment with the AL bias mitigation technique from Richards. 
We will do this on a synthetically biased sample (sample size=700) that we obtained from the CERN dataset, with the rf classifier trained on 10 percent of the data and using a=1/28 and k=0 for the bin distribution

In [50]:
# Some notes:
# - If you use df.loc[i], you get the instance that had index i in the original dataframe.
# - If you use df.l=iloc[i], you get the ith instance n the current dataframe.

In [51]:
import seaborn as sns
import pandas as pd
import numpy as np
#import sys
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
import joblib
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from time import time
import warnings
import os

warnings.filterwarnings(action='ignore')

## Read parameters

In [52]:
#path_to_df = sys.argv[1]
#path_to_df_og = sys.argv[1]
#n_of_iterations = sys.argv[3]
n_of_iterations=50

# Import data

In [53]:
def read_file(url):

    url = url + "?raw=true"
    df = pd.read_csv(url, encoding='cp1252', sep=',', low_memory=False)
    return df

In [54]:
#df = read_file("https://github.com/hannahgathu/Thesis-Data-Visualisations/blob/main/Data/benchmark_data/Test_datasets/bias_and_noise/CERN_testing_set_300_1000.csv")
df = read_file("https://github.com/hannahgathu/Thesis-Data-Visualisations/blob/main/Data/benchmark_data/Test_datasets/bias_and_noise/CERN_testing_set_50_300.csv")
df_og = read_file("https://github.com/hannahgathu/Thesis-Data-Visualisations/blob/main/Data/benchmark_data/dielectron_classification.csv")

In [55]:
df_og = df_og.rename(columns={"M>21.3":"y"})
df = df.rename(columns={"M>21.3":"y"})

## Create sample and pool

In [56]:
sample=df[df['in_biased_sample']==1]
pool=df[df['in_biased_sample']==0]

X=df.drop(columns=['y','M','Run','Event','in_biased_sample'])
y=df.y
X_pool=pool.drop(columns=['y','M','Run','Event','in_biased_sample'])
X_sample = sample.drop(columns=['y','M','Run','Event','in_biased_sample'])
y_pool=pool.y
y_sample=sample.y

In [57]:
X_og=df_og.drop(columns=['y','M','Run','Event'])
y_og=df_og.y

In [58]:
#save starting sample and pool
sample.to_csv('starting_sample')
pool.to_csv('starting_pool')

## Random Forest

In [59]:
##get maximum probabilities
def get_max_class_probabilities(X,rf):
    class_probabilities= rf.predict_proba(X.drop(X_sample.columns[0], axis=1).drop(X_sample.columns[1], axis=1))
    max_class_probabilities=[0]*(len(class_probabilities))
    for i in range (len(class_probabilities)):
        max_class_probabilities[i]=max(class_probabilities[i,0],class_probabilities[i,1])
    return max_class_probabilities

In [60]:
#X=training set, pool=pool of instances to pick from
rf = RandomForestClassifier()
rf.fit(X_sample,y_sample)
#proximity_matrix=get_proximity_matrix(X+pool,rf)

RandomForestClassifier()

## AL query functions

In [61]:
def uncertainty(i):
    return (1- max_class_probabilities[i])
    
def query_instance (X_sample, X_pool, rf):
    max_class_probabilities= get_max_class_probabilities(X, rf)
    #proximity_matrix = get_proximity_matrix(X, rf)
    
    uncertainty_scores=[0]*X_pool.shape[0]
    
    for i in range(X_pool.shape[0]):
        uncertainty_scores[i]= uncertainty(int(X_pool.iloc[i].iloc[0]))
    queried_instance_index = int(X_pool.iloc[np.argmax(uncertainty_scores)].iloc[0])
    return queried_instance_index

    #what I want to do:
    #get all values of s2 for the instances in the pool in a list
    #get the argmax, which gives the index i with the highest s2 score
    #get the instancce from the pool by using 
    #int(X_sample.iloc[0].iloc[0])

## Create folders

In [62]:
os.mkdir('models') 
os.mkdir('query_scores') 

## Create metrics table

In [63]:
metrics_names = ['accuracy', 'balanced_accuracy','recall', 'precision', 'auc', 'pr_auc', 'f1_score', 'f2_score', 'log_loss']
metrics_table = pd.DataFrame(0, index=np.arange(n_of_iterations+1), columns=metrics_names)

In [68]:
def store_metrics (i, y_pred, y_og):
    fpr, tpr, thresholds = metrics.roc_curve(y_og, y_pred)
    precision, recall, thresholds = metrics.precision_recall_curve(y_og, y_pred)
    metrics_list=[metrics.accuracy_score(y_og, y_pred),
                  metrics.balanced_accuracy_score(y_og, y_pred),
                  metrics.recall_score(y_og, y_pred),
                  metrics.precision_score(y_og, y_pred),
                  metrics.auc(fpr, tpr),
                  metrics.auc(recall, precision),
                  metrics.f1_score(y_og, y_pred),
                  metrics.fbeta_score(y_og, y_pred, 2),
                  metrics.log_loss(y_og, y_pred)
                 ]
    metrics_table.loc[i]= metrics_list

## Mitigate bias

In [69]:
sample_size  = sample.shape[0]
pool_size = pool.shape[0]

#run AL algorithm without density criterium
queried_instances=[]
rf = RandomForestClassifier()
t0 = time()

for i in range(n_of_iterations):
    # one iteration of the AL algorithm:
    #retrain classifier
    rf.fit(X_sample.drop(X_sample.columns[0], axis=1).drop(X_sample.columns[1], axis=1), y_sample)

    #store metrics
    y_pred = rf.predict(X_og)
    store_metrics(i, y_pred, y_og)
    
    #save metrics
    metrics_table.to_csv('metrics_table')


    #recalculate class probabilities and proximity matrix
    max_class_probabilities= get_max_class_probabilities(X, rf)
    
    if (i%10 == 0):
        print ('time: ', round(time()-t0, 3), 's')
        #save model
        joblib.dump(rf, './models/{}.joblib'.format(i))
        
    
    #query instance and add to sample
    n = query_instance(X_sample, X_pool, rf)
    X_pool=X_pool.drop(n)
    y_pool=y_pool.drop(n)
    X_sample = X_sample.append(X.loc[n])
    y_sample = pd.concat([y_sample, pd.DataFrame([y.loc[n]])], axis = 0)
    queried_instances.append(n)
    
    #overwrite queried instances
    queried_instances_df=pd.DataFrame(queried_instances)
    queried_instances_df.to_csv('queried_instances')

#store final metrics
y_pred = rf.predict(X_og)
store_metrics(n_of_iterations, y_pred, y_og)
#save metrics
metrics_table.to_csv('metrics_table')

time:  0.79 s
time:  8.969 s
time:  17.355 s
time:  25.753 s
time:  34.434 s


## Save files